# LangGraph Agent with Web Search

This notebook demonstrates a multi-node AI agent using LangGraph that can:
- Decide if a query needs web search
- Perform web searches using Tavily
- Generate intelligent responses based on search results or knowledge

## 1. Install Required Dependencies

Required packages:
- langgraph
- langchain
- langchain-openai
- langchain-community
- tavily-python
- python-dotenv

In [ ]:
# Uncomment to install packages
# !pip install langgraph langchain langchain-openai langchain-community tavily-python python-dotenv

## 2. Import Libraries

In [34]:
import os
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
import operator
from langchain_core.runnables.graph_ascii import draw_ascii 
from dotenv import load_dotenv
from langchain_anthropic import ChatAnthropic

# Load environment variables (set OPENAI_API_KEY and TAVILY_API_KEY first)
load_dotenv()

True

## 3. Define the Agent State

In [35]:
class AgentState(TypedDict):
    messages: Annotated[List[dict], operator.add]  # Accumulates conversation history
    needs_search: bool  # Flag to determine if web search is needed

## 4. Initialize Tools & LLM

In [36]:
# Check for required environment variables
openai_key = os.getenv("OPENAI_API_KEY")
tavily_key = os.getenv("TAVILY_API_KEY")

if not openai_key:
    raise ValueError("OPENAI_API_KEY not found in environment variables. Please set it in your .env file.")
if not tavily_key:
    raise ValueError("TAVILY_API_KEY not found in environment variables. Please set it in your .env file.")

# Initialize LLM and search tool
# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
search_tool = TavilySearchResults(max_results=3)

print("✅ LLM and search tool initialized successfully!")


llm = ChatAnthropic(
    model="claude-opus-4-6",  # or "claude-3-haiku-20240307"
    temperature=0,
    max_tokens=1024
)

✅ LLM and search tool initialized successfully!


## 5. Define Graph Nodes

In [37]:
def should_search_node(state: AgentState) -> AgentState:
    """Decide if we need to search the web based on the query."""
    messages = [
        SystemMessage(
            content="Determine if this query requires current/real-time information "
            "or factual verification that might change over time. "
            "Respond ONLY with 'yes' or 'no'."
        ),
        HumanMessage(content=f"Query: {state['messages'][-1]['content']}")
    ]
    response = llm.invoke(messages)
    needs_search = "yes" in response.content.lower()
    return {"needs_search": needs_search}

In [38]:
def search_node(state: AgentState) -> AgentState:
    """Perform web search and add results to conversation."""
    query = state["messages"][-1]["content"]
    results = search_tool.invoke({"query": query})
    
    # Format search results
    formatted_results = "\n".join([
        f"Result {i+1}: {res['content']}" 
        for i, res in enumerate(results)
    ])
    
    # Add search results as an AI message
    return {
        "messages": [
            {
                "role": "assistant",
                "content": f"Search results for '{query}':\n{formatted_results}"
            }
        ]
    }

In [45]:
def respond_node(state: AgentState) -> AgentState:
    """Generate final response using conversation history."""
    # Extract the original user query (first message)
    user_query = None
    search_results = None
    
    for msg in state["messages"]:
        if msg["role"] == "user":
            user_query = msg["content"]
        elif msg["role"] == "assistant" and "Search results" in msg["content"]:
            search_results = msg["content"]
    
    # Build messages ensuring conversation ends with user message
    messages = [SystemMessage(content="You are a helpful assistant. Answer the user's question based on the provided information.")]
    
    if search_results:
        # Add search results as context
        messages.append(AIMessage(content=search_results))
    
    # Always end with user message
    messages.append(HumanMessage(content=user_query or state["messages"][-1]["content"]))
    print(messages)
    response = llm.invoke(messages)
    return {
        "messages": [{"role": "assistant", "content": response.content}]
    }

## 6. Define Conditional Routing

In [40]:
def route_after_decision(state: AgentState) -> str:
    """Route to search or direct response based on decision."""
    return "search" if state["needs_search"] else "respond"

## 7. Build the Graph

In [41]:
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("decide", should_search_node)
workflow.add_node("search", search_node)
workflow.add_node("respond", respond_node)

# Set entry point
workflow.set_entry_point("decide")

# Add edges with conditional routing
workflow.add_conditional_edges(
    "decide",
    route_after_decision,
    {"search": "search", "respond": "respond"}
)
workflow.add_edge("search", "respond")
workflow.add_edge("respond", END)

# Compile the graph
app = workflow.compile()

In [43]:
# Visualize the graph
from langchain_core.runnables.graph_ascii import draw_ascii 
print("📊 Agent Graph Structure:")
print("=" * 50)

# Print ASCII representation
print(app.get_graph().draw_ascii())

📊 Agent Graph Structure:
         +-----------+      
         | __start__ |      
         +-----------+      
               *            
               *            
               *            
          +--------+        
          | decide |        
          +--------+        
          ..        ..      
        ..            ..    
       .                ..  
+--------+                . 
| search |              ..  
+--------+            ..    
          **        ..      
            **    ..        
              *  .          
          +---------+       
          | respond |       
          +---------+       
               *            
               *            
               *            
          +---------+       
          | __end__ |       
          +---------+       


## 8. Run the Agent

In [46]:
# Example 1: Question needing search
print("🔍 Example 1: Current events question")
inputs = {"messages": [{"role": "user", "content": "What were the major AI announcements at CES 2026?"}], "needs_search": False}
result = app.invoke(inputs)
print("Answer:", result["messages"][-1]["content"])
print("\n" + "="*50 + "\n")

🔍 Example 1: Current events question

Answer: # Major AI Announcements at CES 2026

CES 2026 (January 6-9, Las Vegas) marked a significant shift for AI, moving from digital interfaces into the **physical world** through physical AI and proactive agents. Here are the key announcements:

## NVIDIA's Rubin Platform
The biggest announcement was **NVIDIA's launch of the Vera Rubin architecture**, replacing the Blackwell series. Key highlights include:
- A suite of **six new chips**, including the Vera CPU and Rubin GPU
- **5x the performance** of Blackwell
- **10x reduction** in inference token costs
- **4x fewer GPUs** needed to train Mixture-of-Experts (MoE) models
- A new **Inference Context Memory Storage platform** using BlueField-4 processors to accelerate multistep agentic reasoning
- Major cloud providers (**Microsoft, AWS, Google, CoreWeave**) and partners (Cisco, Dell, HPE, Lenovo) committed to deploying Rubin-based superfactories in the second half of 2026

## AMD's Data Center P

In [47]:
# Example 2: Question answerable without search
print("💡 Example 2: General knowledge question")
inputs = {"messages": [{"role": "user", "content": "Explain how photosynthesis works"}], "needs_search": False}
result = app.invoke(inputs)
print("Answer:", result["messages"][-1]["content"])

💡 Example 2: General knowledge question

Answer: # Photosynthesis

Photosynthesis is the process by which plants, algae, and some bacteria convert light energy into chemical energy (glucose), using carbon dioxide and water. It is fundamental to life on Earth.

## Overall Equation

**6CO₂ + 6H₂O + light energy → C₆H₁₂O₆ + 6O₂**

## The Two Main Stages

### 1. Light-Dependent Reactions (in the thylakoid membranes)
- **Light absorption:** Chlorophyll and other pigments in photosystems (PS II and PS I) capture sunlight.
- **Water splitting (photolysis):** Water molecules are split into hydrogen ions (H⁺), electrons, and **oxygen** (released as a byproduct).
- **Electron transport chain:** Excited electrons pass through a series of proteins, releasing energy used to pump H⁺ ions across the membrane.
- **ATP & NADPH production:** The H⁺ gradient drives ATP synthase to produce **ATP**, and electrons ultimately reduce NADP⁺ to form **NADPH**.

### 2. Light-Independent Reactions / Calvin Cycle 